In [ ]:
import torch
from torchvision import datasets, transforms
import torchvision
from torchvision import models
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt
import os
from torch import nn
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import random

In [ ]:
'''SETTING UP U-NET NEURAL NETWORK'''

class Block(nn.Module):
    '''
    Block has three layers: two convolution operators (no padding)
    and one with ReLU activation
    '''
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 5, padding='same')
        self.relu  = nn.ReLU()
        self.conv2 = nn.Conv2d(out_ch, out_ch, 5, padding='same')        
    
    def forward(self, x):
        return self.conv2(self.relu(self.conv1(x)))


class Encoder(nn.Module):
    '''
    Encoder: Maxpooling (used for downsampling) performed between two Block operations
    This class includes the block operations already
    '''
    def __init__(self, chs=(1,64,128,256,512,1024)):
        super().__init__()
        self.enc_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs)-1)])
        self.pool       = nn.MaxPool2d(2)
    def forward(self, x):
        ftrs = []
        for block in self.enc_blocks:
            x = block(x)
            ftrs.append(x)
            x = self.pool(x)
        return ftrs


class Decoder(nn.Module):
    '''
    Decoder: Series of upconvolutions and decoders to expand matrix in order to determine location
    '''
    def __init__(self, chs=(1024, 512, 256, 128, 64)):
        super().__init__()
        self.chs         = chs
        self.upconvs    = nn.ModuleList([nn.ConvTranspose2d(chs[i], chs[i+1], 2, 2) for i in range(len(chs)-1)])
        self.dec_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs)-1)]) 
        
    def forward(self, x, encoder_features):
        for i in range(len(self.chs)-1):
            x        = self.upconvs[i](x)
            enc_ftrs = self.crop(encoder_features[i], x)
            x        = torch.cat([x, enc_ftrs], dim=1)
            x        = self.dec_blocks[i](x)
        return x
    
    def crop(self, enc_ftrs, x):
        _, _, H, W = x.shape
        enc_ftrs   = transforms.CenterCrop([H, W])(enc_ftrs)
        return enc_ftrs


class UNet(nn.Module):
    '''
    Code to set up encoders and decoder layers
    '''
    def __init__(self, enc_chs=(1,2,4,8,16,32), dec_chs=(32, 16,8, 4, 2), num_class=3, retain_dim=True, out_sz=(1024,1024)):
        super().__init__()
        self.encoder     = Encoder(enc_chs)
        self.decoder     = Decoder(dec_chs)
        self.head        = nn.Conv2d(dec_chs[-1], num_class, 1)
        #self.retain_dim  = retain_dim

    def forward(self, x, out_sz=(256,256)):
        enc_ftrs = self.encoder(x)
        out      = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
        out      = self.head(out)
        
        #if self.retain_dim:
            #out = nn.functional.interpolate(out, out_sz)
            
        return out

In [ ]:
'''SETUP DATASETS'''

def normalization(x):
    '''
    Used to normalize sample data so that it is between 0 and 1.
    This is only used on the input image, not the mask
    '''
#     x_max = torch.max(x)
#     x_min = torch.min(x)
#     x_norm = (x-x_min)/(x_max-x_min)
    
    x_mean = torch.mean(x)
    x_stdev = torch.std(x)
    x_norm = (x-x_mean)/(x_stdev)
    return x_norm

def rand_input(x, **kwargs):
    '''
    This function randomizes the mask values and replaces the original values with random
    numbers between 0 and 1. This function was created in order to bypass the thresholding
    we experienced during initial tests. (The model categorized on intensity rather than shape)
    The mask values on the input are always 0 for background, 1 for streaks, 2 for beam stop
    '''
    x = x.float().to(device)
    val = torch.rand(3).to(device) #Generate three random numbers and assign to each value 
    x = torch.where(x==0,val[0],x)
    x = torch.where(x==1, val[1],x)
    x = torch.where(x==2, val[2],x)
    return x


class Dataset(Dataset):
#'Characterizes a dataset for PyTorch'
    def __init__(self, list_IDs, labels):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        
        # In order to handle different indexing for validation and training data
        while True:
            try:
                ID = self.list_IDs[index]
                break
            except:
                index += len(self.list_IDs)
                
        # Load data and get label
        X = torch.load(ID)
        plt.imshow(X["target"])
        plt.title("Raw image")
        plt.show()
#         plt.hist(X['target'][:,0].cpu().detach().numpy().flatten())
#         plt.show()

        #Randomize
        x_rand = rand_input(X['target']).cpu()
#         plt.imshow(x_rand)
#         plt.title("Image with values randomized")
#         plt.show()
        #x_rand += 0.1*torch.rand(x_rand.shape)

        X['input'] = normalization(x_rand)
        return X

In [ ]:
'''FUNCTIONS'''

def cuda_setup():
    # CUDA for PyTorch
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")
    
    torch.backends.cudnn.benchmark = True
    print("Using {} device".format(device))
    
    return(device)

def set_rand_seed(seed):
    '''
    Creates random seed for each library so that randomness is repeatable. 
    Initialized first to set all randomness
    '''
    
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

def save_params(output_array, orig_array, params, desc, loss, model, optimizer, i):
    '''
    Saves what I have changed in each iteration
    
    Inputs:
        Long string of output graphs
        List of params
        Comments of what was changed. 
    Outputs: 
        Pytorch file which lists all of these in a dictionary
        
    TODO: This function is no where near complete. Will need to look at this in-depth
    '''
    
    output_tensor = torch.Tensor(output_array)
    orig_tensor = torch.Tensor(orig_array)
    
    torch.save({
        "output": output_tensor, 
        "original": orig_tensor, 
        "params":params, 
        "description": desc, 
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss}, 
        f"nsls2/users/maire1/unet/params_list{i}.pt")
    i += 1
    return i

def calculate_weights(target):
    '''
    Calculates the weight which should be assigned based off of the target.
    This is done to give more importance to values which are not represented equally 
    amount-wise. 
    
    Inputs: Target array
    Outputs: Array of weights for each value
    '''
    
    #Find the amount of pixels which belong to each category 
    total = target.numel()
    stop = target.eq(2).sum().item()/2
    streaks = target.eq(1).sum().item()
    bkgnd = total - streaks - stop
    
    #Need to have a lot of edge cases in case if there are no streaks, beamstops, etc. in the image
    if streaks == 0 and stop == 0:
        weights = [1, 1, 1]
    elif streaks == 0:
        weights = [1, 1, bkgnd/stop]
    elif stop == 0:
        weights = [1, bkgnd/streaks, 1]
    else:
        weights = [1, bkgnd/streaks, bkgnd/stop]
    return weights

In [ ]:
def train(dataloader, model, optimizer, device, training_graphs, orig_images):
    '''
    This function can likely be split into smaller functions. The training function will
    analyze the training data and output the loss. 
    
    Note: There are a lot of things commented out. Some lines which are commented out
    were used for simulation data, and they do not work on real data. Other lines 
    were commented out based on whether we are using target data as the input or not. 
    Each line will have a marker explaining what it is for (Toy Data or Target or Real)'''
    
    size = len(dataloader)
    model.train()
    loss_tot = np.zeros(size)
    
    for batch_num, local_batch in enumerate (dataloader):

        # Transfer to GPU
        for key, value in local_batch.items():
#             local_batch[key] = value.to(device) #Toy data
            local_batch[key] = value.float().to(device) #REAL DATA CHANGE
#        in_batch = local_batch['input'].unsqueeze(1) #Using image as input
        in_batch = local_batch['target'].unsqueeze(1) #Using target as input
        
        #Run input through the model
        out  = model(in_batch)
        
        target = local_batch['target']
        
        #WEIGHTS
        weight = calculate_weights(target)
        class_weight = torch.FloatTensor(weight).to(device)
        loss_fn = nn.CrossEntropyLoss(class_weight)
        loss = loss_fn(out, target.long())

#         #GRAPHING PT 1
          #There are more opportunities to see the output beyond this (see end of code)
          #This graphing outputs thousands of images. 
#         nn_image = out.argmax(1)
#         nn_image = nn_image[0,:,:].cpu().numpy()
#         training_graphs.append(nn_image)
        
#         y_graph = in_batch[0,0,:,:].cpu().numpy()
#         orig_images.append(y_graph)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss = loss.cpu().detach()
        
        loss, current = loss.item(), batch_num #* len(out)
        # print(f"loss: {loss:>7f}  [{(current+1):>5d}/{size:>5d}]")
        loss_tot[batch_num] = loss
        
        if torch.cuda.torch.cuda.is_available():
            torch.cuda.empty_cache()
            
    mean = np.mean(loss_tot)
    print("Train Mean Loss: ", mean)
    

    #GRAPHING CTD.
#     for i in range (len(training_graphs)):
#         plt.imshow(training_graphs[i])
#         plt.show()

#         plt.imshow(orig_images[i])
#         plt.show()

    return training_graphs, orig_images, loss, optimizer
    
                
def validation(dataloader, model, device):
    '''
    This function is largely similar to the training function.
    This will display the loss and accuracy
    
    TODO: Fix accuracy output. '''
    
    model.eval()
    test_loss, correct = 0, 0 #This adds throughout the process
    
    with torch.no_grad():
        for batch_num, local_batch in enumerate (dataloader):
            # Transfer to GPU
            for key, value in local_batch.items():
#                 local_batch[key] = value.to(device) #Toy data
                local_batch[key] = value.float().to(device) #FOR REAL DATA
                
            #pred = model(local_batch['input'].unsqueeze(1)) #Image as input
            y = local_batch['target']
            pred = model(y.unsqueeze(1)) #Target as input
            
            #WEIGHTS
            weight = calculate_weights(y)
            class_weight = torch.FloatTensor(weight).to(device)
            loss_fn = nn.CrossEntropyLoss(class_weight) 
            
            test_loss += loss_fn(pred, y.long())#.detach()
            
            correct += (pred.argmax(axis=1) == y).type(torch.float).sum()
            #correct += (pred.argmax(1) == y).type(torch.float).sum()#.detach()
            tot_pts = torch.numel(y)
            
        #Calculating test loss by dividing total loss over batch size
        size = batch_num + 1
        test_loss /= size
        #print ("Correct:", correct.shape(), "\nTotal Size: ", size*tot_pts)
        correct /= (size*tot_pts)
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
        
        if torch.cuda.torch.cuda.is_available():
            torch.cuda.empty_cache() 

In [ ]:
#PARAMS
params = {'batch_size': 4, #Number of samples in each batch
          'shuffle': True,
          'num_workers': 0} #Number of process to generate batches in //
max_epochs = 33 
learning_rate = 1e-3
training_num = 160 #This is an arbitrary number
num = 0 #FOR RANDOM SEED

In [ ]:
set_rand_seed(num)

#Init
output_array = []
orig_array = []
#desc = "First saving run. Only trained on the mask input" #For future model saving

#Training
#datafile =  "/nsls2/users/maire1/sim_data/sim_address.csv" #For simulation data
datafile = "/nsls2/users/maire1/unet/data/cropped_data/data_address.csv" 
  
# Determine and setup device
device = cuda_setup()

# Datasets
# Separating datasets into training and validation. 
file = pd.read_csv(datafile)
partition = file['address']# IDs
training_partition = partition[:training_num]
validation_partition = partition[training_num:]

labels = file['sample']# Labels
training_labels = labels[:training_num]
validation_labels = labels[training_num:]

model = UNet().to(device) #comment this out when you start training

# Generators
training_set = Dataset(training_partition, training_labels)
training_generator = DataLoader(training_set, **params, pin_memory=True)

validation_set = Dataset(validation_partition, validation_labels)
validation_generator = DataLoader(validation_set, **params, pin_memory=True)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


# Loop over epochs
for epoch in range(max_epochs):
    # Training
    print(f"Epoch {epoch+1}\n-------------------------------")
    output_array, orig_array, loss, optimizer = train(training_generator, model, optimizer, device, training_graphs=output_array, orig_images=orig_array)
    validation(validation_generator, model, device)
    
#i = save_params(output_array, orig_array, params, desc, loss, model, optimizer)
    

In [ ]:
# FOR GRAPHING A FEW OUTPUTS

In [ ]:
local_batch = next(iter(training_generator))
x = local_batch['input'].unsqueeze(1).to(device)
# x_r = rand_input(x)
# x_norm = normalization(x_r)
x_hat = model(x)

In [ ]:
fig, ax = plt.subplots(1, 3, dpi = 200)

k =2

ax[0].imshow(x[k, 0, :, :].cpu().detach().numpy())#, vmin = -1, vmax = 1)
ax[1].imshow(x_hat[k, :, :, :].cpu().detach().numpy().argmax(0))#, vmin = 0, vmax = 2)
#ax[1].imshow(x_hat[k, 1, :, :].cpu().detach().numpy(), vmin = 0, vmax = 2) 
ax[2].imshow(local_batch['target'][k])
for j in range(3):
    ax[j].axis('off')
    
ax[0].set_title('input')
ax[1].set_title('model output')
ax[2].set_title('target')

In [ ]:
plt.hist(x[2,0,:,:].cpu().detach().numpy().flatten())

